In [1]:
import wobble
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import tensorflow as tf

/Users/mbedell/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=np.arange(72))
results = wobble.Results(data=data)

In [3]:
for r in reversed(range(data.R)):
    model = wobble.Model(data, results, r)
    model.add_star('star')
    model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)
    print("--- ORDER {0} ---".format(r))
    with tf.Session() as session:
        with session.as_default():
            wobble.optimize_order(model, niter=80)
    del model
    #if (r % 5) == 0:
    #    results.write('results_order{0}.hdf5'.format(data.orders[r]))

--- ORDER 71 ---


100%|██████████| 80/80 [00:01<00:00, 50.68it/s]


--- ORDER 70 ---


100%|██████████| 80/80 [00:01<00:00, 50.67it/s]


--- ORDER 69 ---


...

100%|██████████| 80/80 [00:02<00:00, 39.62it/s]


--- ORDER 43 ---


100%|██████████| 80/80 [00:02<00:00, 38.96it/s]


--- ORDER 42 ---


KeyboardInterrupt: 

In [ ]:
results.component_names

In [ ]:
results.write('test.hdf5')